In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import asyncio

from computer_use_demo.loop import sampling_loop, PROVIDER_TO_DEFAULT_MODEL_NAME, APIProvider, SYSTEM_PROMPT
import os

from playwright.async_api import async_playwright

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page(viewport={"width": 1024, "height": 768})

In [ ]:
messages = []

In [ ]:
from print_utils import print_conversation, print_stuff, noop
            
async def converse():
    await sampling_loop(
            model=PROVIDER_TO_DEFAULT_MODEL_NAME[APIProvider.ANTHROPIC],
            api_key=os.environ.get("ANTHROPIC_API_KEY"),
            provider=APIProvider.ANTHROPIC,
            messages=messages,
            system_prompt_suffix="",
            output_callback=print_stuff,
            tool_output_callback=print_stuff,
            api_response_callback=noop,
            page=page,
        )
    await asyncio.sleep(0.1)

In [ ]:
user_message = input()
if len(user_message) > 0:
    messages.append({
        "role": "user",
        "content": user_message
    })
    await converse()
    print_conversation(messages)

In [ ]:
print_conversation(messages)

In [ ]:
await browser.close()
await playwright.stop()